In [62]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc

In [63]:
import cx_Oracle
import pandas as pd
import numpy as np
# import db_config

user = "Risk"
pw = "Risk123"
connect_string = "udwh-prod1.msk.russb.org:1521/udwhprod.msk.russb.org"

con = cx_Oracle.connect(user, pw, connect_string)
print("Database version:", con.version)

Database version: 12.2.0.1.0


In [64]:
cur = con.cursor()

In [65]:
 zapros = \
"  \
select SCORE_OKB,SCORE_EQUIFAX,a.REQ_AT,b.* from kuz_big_table a \
left join GL_ALL b \
on to_char(a.req_id) = to_char(b.req_id) \
where PRODUCT_ID ='КН' \
" 

In [66]:
 zapros

"  select SCORE_OKB,SCORE_EQUIFAX,a.REQ_AT,b.* from kuz_big_table a left join GL_ALL b on to_char(a.req_id) = to_char(b.req_id) where PRODUCT_ID ='КН' "

In [67]:
cur.execute(zapros)

<cx_Oracle.Cursor on <cx_Oracle.Connection to Risk@udwh-prod1.msk.russb.org:1521/udwhprod.msk.russb.org>>

In [68]:
df=pd.DataFrame(cur.fetchall())

In [69]:
num_fields = len(cur.description)
df.columns = [i[0] for i in cur.description]

In [70]:
df

SCORE_OKB SCORE_EQUIFAX     REQ_AT   REQ_ID  GL30_3  GL60_3  GL90_3  \
0             892           659 2019-10-01  3022409     0.0     0.0     0.0   
1             739           653 2019-10-09  3028461     0.0     0.0     0.0   
2             805           705 2019-10-08  3027456     0.0     0.0     0.0   
3             818           606 2019-10-10  3029972     0.0     0.0     0.0   
4             912           778 2019-10-11  3030966     0.0     0.0     0.0   
...           ...           ...        ...      ...     ...     ...     ...   
1802219        -1            -1 2020-01-18     None     NaN     NaN     NaN   
1802220      None          None 2020-11-11     None     NaN     NaN     NaN   
1802221        -1            -1 2020-12-03     None     NaN     NaN     NaN   
1802222       877           634 2020-12-21     None     NaN     NaN     NaN   
1802223      None          None 2021-06-04     None     NaN     NaN     NaN   

         GL30_6  GL30_5  GL60_6  GL90_6  GL30_9  GL60_9  GL90_9  GL30_12  \
0           0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0   
1           0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0   
3           1.0     1.0     1.0     1.0     1.0     1.0     1.0      1.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0   
...         ...     ...     ...     ...     ...     ...     ...      ...   
1802219     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN   
1802220     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN   
1802221     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN   
1802222     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN   
1802223     NaN     NaN     NaN     NaN     NaN     NaN     NaN      NaN   

         GL60_12  GL90_12  
0            0.0      0.0  
1            0.0      0.0  
2            0.0      0.0  
3            1.0      1.0  
4            0.0      0.0  
...          ...      ...  
1802219      NaN      NaN  
1802220      NaN      NaN  
1802221      NaN      NaN  
1802222      NaN      NaN  
1802223      NaN      NaN  

[1802224 rows x 17 columns]

In [71]:
df=df.drop('REQ_ID',axis=1)

In [72]:
df['PERIOD']=df['REQ_AT'].dt.strftime('%Y-%m')
df=df.drop('REQ_AT',axis=1)

In [73]:
df=df.iloc[:10000]

In [74]:
filter_col = [col for col in df.columns if col.startswith('GL')]
filter_col

['GL30_3',
 'GL60_3',
 'GL90_3',
 'GL30_6',
 'GL30_5',
 'GL60_6',
 'GL90_6',
 'GL30_9',
 'GL60_9',
 'GL90_9',
 'GL30_12',
 'GL60_12',
 'GL90_12']

In [75]:
import warnings
warnings.filterwarnings("ignore")

In [76]:
# СКОРЫ ДОЛЖНЫ БЫТЬ В ФОРМАТЕ ЧИСЛО И ХУДШИЕ СВЕРХУ
df['SCORE_OKB']=pd.to_numeric(df['SCORE_OKB'], errors='coerce')*-1
df['SCORE_EQUIFAX']=pd.to_numeric(df['SCORE_EQUIFAX'], errors='coerce')*-1

In [77]:
df.dtypes

SCORE_OKB        float64
SCORE_EQUIFAX    float64
GL30_3           float64
GL60_3           float64
GL90_3           float64
GL30_6           float64
GL30_5           float64
GL60_6           float64
GL90_6           float64
GL30_9           float64
GL60_9           float64
GL90_9           float64
GL30_12          float64
GL60_12          float64
GL90_12          float64
PERIOD            object
dtype: object

In [78]:
from sklearn.metrics import roc_curve, auc
df_auc=[]
c=0
for i in ('SCORE_EQUIFAX','SCORE_OKB'):
    for z in filter_col:
        for k in set(df['PERIOD']):
            c+=1
            print(c)
            try:
#                 print(i,z,k)
                data=df[(df.PERIOD == k)][[i,z]].dropna()
                fpr, tpr, _ = roc_curve(data.iloc[:,1], data.iloc[:,0])
                cal_auc=auc(fpr, tpr)*2-1
                df_auc.append([i,z,k,cal_auc])
#                 print(df_auc)
            except:
#                 print(i,z,k,'continue')
                continue
            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [79]:
df_ss=pd.DataFrame(df_auc,columns=['SCORE', 'GL', 'DATASET','GINI'])
# df_ss.columns=['a','b','c','d']
# df_ss['#']=df_ss['SCORE'].astype(str)+'_'+df_ss['DATASET'].astype(str)
df_ss['#']=df_ss['SCORE'].astype(str)+'_'+df_ss['GL'].astype(str)
# df_ss=df_ss.drop(columns=['SCORE', 'DATASET'])
df_ss = pd.pivot_table(df_ss, values='GINI', index=['#'],
                    columns=['DATASET'], aggfunc=np.sum, fill_value=0)
df_ss=df_ss.reset_index()
# df_ss=df_ss.drop(columns=['SSS'])
df_ss

DATASET                      #   2019-09   2019-10   2019-11   2019-12  \
0        SCORE_EQUIFAX_GL30_12  0.000000  0.444231  0.384590  0.300312   
1         SCORE_EQUIFAX_GL30_3  0.000000  0.249485  0.384461  0.353775   
2         SCORE_EQUIFAX_GL30_5  0.000000  0.460832  0.347406  0.302401   
3         SCORE_EQUIFAX_GL30_6  0.000000  0.437452  0.362111  0.203160   
4         SCORE_EQUIFAX_GL30_9  0.000000  0.395161  0.432446  0.279576   
5        SCORE_EQUIFAX_GL60_12  0.000000  0.367422  0.353314  0.283469   
6         SCORE_EQUIFAX_GL60_3  0.000000  0.373737  0.481481  0.225333   
7         SCORE_EQUIFAX_GL60_6  0.000000  0.411696  0.326440  0.310939   
8         SCORE_EQUIFAX_GL60_9  0.000000  0.321921  0.411066  0.238207   
9        SCORE_EQUIFAX_GL90_12  0.000000  0.411765  0.411270  0.266596   
10        SCORE_EQUIFAX_GL90_3  0.000000  0.000000  0.000000  0.000000   
11        SCORE_EQUIFAX_GL90_6  0.000000  0.325561  0.310133  0.348024   
12        SCORE_EQUIFAX_GL90_9  0.000000  0.367659  0.385132  0.220864   
13           SCORE_OKB_GL30_12  0.333333  0.183654  0.290542  0.257643   
14            SCORE_OKB_GL30_3  0.000000  0.089691  0.208429  0.357085   
15            SCORE_OKB_GL30_5  0.000000  0.211315  0.269693  0.317158   
16            SCORE_OKB_GL30_6  0.000000  0.172201  0.242293  0.235147   
17            SCORE_OKB_GL30_9  0.333333  0.160906  0.303337  0.243324   
18           SCORE_OKB_GL60_12  0.333333  0.165679  0.237797  0.224894   
19            SCORE_OKB_GL60_3  0.000000  0.494949  0.863946  0.199238   
20            SCORE_OKB_GL60_6  0.000000  0.231579  0.231950  0.327527   
21            SCORE_OKB_GL60_9  0.000000  0.170543  0.257817  0.242958   
22           SCORE_OKB_GL90_12  0.333333  0.202231  0.228924  0.261116   
23            SCORE_OKB_GL90_3  0.000000  0.000000  0.000000  0.000000   
24            SCORE_OKB_GL90_6  0.000000  0.132988  0.191694  0.363864   
25            SCORE_OKB_GL90_9  0.000000  0.137717  0.226895  0.247275   

DATASET   2020-01   2020-02   2020-03   2020-04   2020-05  ...   2020-07  \
0        0.241805  0.302994  0.242915  0.159091  0.714286  ...  0.149470   
1        0.400510 -0.021404 -0.119497  0.000000  0.000000  ...  0.209407   
2        0.180589  0.234035 -0.091880  0.208333  0.000000  ...  0.260030   
3        0.191702  0.190264  0.050691  0.208333  1.000000  ...  0.182107   
4        0.245215  0.248424  0.160232  0.159091  0.714286  ...  0.159535   
5        0.259951  0.320631  0.132239  0.159091  0.714286  ...  0.134041   
6        0.323734  0.002621  0.000000  0.000000  0.000000  ...  0.342529   
7        0.138370  0.291091 -0.091880  0.208333  0.000000  ...  0.284650   
8        0.245629  0.252158  0.023965  0.362319  1.000000  ...  0.183017   
9        0.246693  0.316371 -0.011439  0.362319  0.714286  ...  0.154824   
10       0.000000  0.000000  0.000000  0.000000  0.000000  ...  0.000000   
11       0.376054  0.277368 -0.039490  0.000000  0.000000  ...  0.290162   
12       0.223111  0.226940  0.000000  0.208333  1.000000  ...  0.151109   
13       0.175627  0.307872  0.304469  0.276923  0.470588  ...  0.149031   
14       0.407823  0.393206 -0.127946  0.000000  0.000000  ...  0.050651   
15       0.153268  0.224220  0.024786  0.000000  0.473684  ...  0.091913   
16       0.150944  0.199059  0.047865  0.166667  0.305556  ...  0.171076   
17       0.183094  0.251727  0.133108  0.276923  0.470588  ...  0.123211   
18       0.174742  0.260500  0.168581  0.276923  0.470588  ...  0.119581   
19       0.372986  0.406205  0.000000  0.000000  0.000000  ...  0.224425   
20       0.122828  0.197807  0.024786  0.000000  0.473684  ...  0.134293   
21       0.164312  0.191073 -0.032461  0.129630  0.305556  ...  0.180121   
22       0.162713  0.250824  0.013668  0.129630  0.470588  ...  0.113951   
23       0.000000  0.000000  0.000000  0.000000  0.000000  ...  0.000000   
24       0.318873  0.267606 -0.052041  0.000000  0.000000  ...  0.119453   
25       0.151667  0.1510